In [10]:
import pandas as pd
df = pd.read_csv("/Users/kartashoffv/Documents/rutube-ai-assistant/data/concat_data.csv")
df['concat'] = "Тема запроса: " + df['Классификатор 1 уровня'] + "\n" + "Запрос: " + df['Вопрос из БЗ'] + "\n" + "Ответ: " + df['Ответ из БЗ']

df.head()

,Ответ из БЗ,Вопрос из БЗ,Классификатор 1 уровня,Классификатор 2 уровня,concat
0,1. Зарегистрируйтесь или авторизуйтесь на RUTU...,Как загрузить видеоролик на сайт?,ВИДЕО,Загрузка видео,Тема запроса: ВИДЕО\nЗапрос: Как загрузить вид...
1,RUTUBE Детям пока недоступен в виде отдельного...,Для каких телевизоров и ТВ-приставок доступно ...,ПРЕДЛОЖЕНИЯ,Персонализация 0,Тема запроса: ПРЕДЛОЖЕНИЯ\nЗапрос: Для каких т...
2,RUTUBE не предоставляет сервис по копированию ...,Могу ли я скачать копию моей трансляции?,ТРАНСЛЯЦИЯ,Управление трансляцией,Тема запроса: ТРАНСЛЯЦИЯ\nЗапрос: Могу ли я ск...
3,"RUTUBE предоставляет удобную технологию, интег...",Правила размещения субтитров,ВИДЕО,Загрузка видео,Тема запроса: ВИДЕО\nЗапрос: Правила размещени...
4,Shorts автоматически открываются в вертикально...,Можно ли включить ролик Shorts в горизонтально...,ВИДЕО,Воспроизведение видео,Тема запроса: ВИДЕО\nЗапрос: Можно ли включить...


In [11]:
import chromadb
from chromadb.api.types import EmbeddingFunction, QueryResult


class Chroma:
    def __init__(
        self,
        embedding: EmbeddingFunction,
        collection_name: str,
        path: str = './vector_db_bge_3'
    ) -> None:
        """
        Инициализирует класс для работы с ChromaDB. Используется для первоначальной загрузки данных в БД.
        Args:
            embedding (EmbeddingFunction): функция эмбеддинга.
            collection_name (str): название коллекции в ChromaDB.

        """
        self.embedding = embedding
        self.collection_name = collection_name
        self.chroma_client = chromadb.PersistentClient(path=path)

        self.collection = self.chroma_client.get_or_create_collection(
            name=self.collection_name,
            embedding_function=self.embedding,
            metadata={"hnsw:space": "cosine"} # доступны по дефолту l2 (Squared L2), Inner product (ip), cosine (Cosine similarity)  
        )

    def add_docs(self, data_dict: dict) -> None:
        """
        Adds documents to the ChromaDB.
        
        Args:
            data_dict (dict): Dictionary representing the documents. 
                                Must contain 'docs', 'sources', 'questions', and 'ids' fields.
        """
        docs = data_dict.get("docs")
        sources = data_dict.get("sources")
        ids = data_dict.get("ids")

        if not all([docs, sources, ids]):
            raise ValueError("data_dict must contain 'docs', 'sources' and 'ids' fields.")

        metadatas = [
            {"source": sources[i]} for i in range(len(docs))
        ]

        # Batch processing, чтобы все доки за один раз не обрабатывать 
        for start in range(0, len(docs), 100):
            end = start + 100
            self.collection.add(
                documents=docs[start:end],
                metadatas=metadatas[start:end],
                ids=ids[start:end],
            )

    def get_relevant_docs(self, user_query: str):
        """
        Получает документы из ChromaDB, удовлетворяющие пользовательскому запросу.
        Args:
            query (str): пользовательский запрос.
            collection_name (str): название коллекции ChromaDB, где находятся документы.
            embedding_info (dict): информация об эмбеддинге.
        Returns:
            Объект QueryResult с аттрибутами metadatas, documents и distances.
        """
        # user_query = self.embedding(user_query)
                
        if self.collection.count() > 0:
            return self.collection.query(
                query_texts=user_query,
                include=["metadatas", "documents", "distances"],
                n_results=5,
            )
        else:
            return collection.query(
                query_texts=query,
                include=["metadatas", "documents", "distances"],
                n_results=1,
            )

In [12]:
import chromadb.utils.embedding_functions as embedding_functions
embedding = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="deepvk/USER-bge-m3")

df['ids'] = [f"id_{i}" for i in range(len(df))]
df['sources'] = [f"link_{i}" for i in range(len(df))]

test_data = {
    "docs": df["concat"].tolist(),
    "sources": df["sources"].tolist(),
    "ids": df["ids"].tolist()
}

a = Chroma(
    embedding = embedding, 
    collection_name='bge_concated_3'
)
a.add_docs(test_data)

In [13]:
new_df = pd.read_csv("query_answer.csv")
new_df = new_df[['Вопрос пользователя','Ответ из БЗ']]
new_df.head()

,Вопрос пользователя,Ответ из БЗ
0,Здравствуйте! Можно уточнить причины Правилhtt...,Чужой контент без разрешения автора или правоо...
1,"Добрый вечер, какой топ причин блокировки виде...",Чужой контент без разрешения автора или правоо...
2,"Все пишут, что монетизация на рутубе отключает...","Монетизация может отключиться, если на вашем к..."
3,Что запрещено в монетизации и что можно выклад...,"Монетизация может отключиться, если на вашем к..."
4,"Чтобы не отключали монетизацию, надо, чтобы я ...","Монетизация может отключиться, если на вашем к..."


In [14]:
new_columns = {'chunks': []}

for idx, row in new_df.iterrows():
    result = a.get_relevant_docs(user_query=row['Вопрос пользователя'])
    
    documents = result['documents'][0]
    
    new_columns['chunks'].append(documents)    

new_df['chunks'] = new_columns['chunks']  


In [15]:
accrs = []
def chunk_accrc(row):
    chunks = row['chunks']
    acrc = 0
    for chunk in chunks:
        if row['Ответ из БЗ'] in chunk:
            acrc = 1
    row['accuracy'] = acrc
    return row

new_df = new_df.apply(chunk_accrc, axis=1)
new_df['accuracy'].mean()

np.float64(0.8801498127340824)

In [16]:
import math
import pandas as pd

def compute_ndcg(row, k=None):
    """
    Вычисляет NDCG для одной строки датафрейма.

    Parameters:
    row (pd.Series): Строка датафрейма, содержащая 'Ответ из БЗ' и 'chunks'.
    k (int, optional): Позиция ранга, до которой рассчитывается NDCG.

    Returns:
    float: Оценка NDCG для строки.
    """
    ground_truth = row['Ответ из БЗ']
    retrieved_chunks = row['chunks']

    if k is None:
        k = len(retrieved_chunks)

    # Вычисляем релевантность для каждого чанка
    rels = [1 if ground_truth in chunk else 0 for chunk in retrieved_chunks[:k]]

    # Вычисляем DCG
    dcg = 0.0
    for i, rel in enumerate(rels):
        dcg += rel / math.log2(i + 2)

    # Вычисляем IDCG (идеальный DCG)
    ideal_rels = sorted(rels, reverse=True)
    idcg = 0.0
    for i, rel in enumerate(ideal_rels):
        idcg += rel / math.log2(i + 2)

    ndcg = dcg / idcg if idcg > 0 else 0.0
    return ndcg

# Применяем функцию к каждому ряду датафрейма
new_df['ndcg'] = new_df.apply(compute_ndcg, axis=1)
new_df['ndcg'].mean()

np.float64(0.7643072965098522)